In [1]:
# !pip install torchaudio==0.9.0
import torch
import torchaudio
import numpy as np
from tqdm import tqdm

## 0. 데이터셋 다운로드

In [3]:
!wget http://mi.soi.city.ac.uk/datasets/magnatagatune/mp3.zip.001
!wget http://mi.soi.city.ac.uk/datasets/magnatagatune/mp3.zip.002
!wget http://mi.soi.city.ac.uk/datasets/magnatagatune/mp3.zip.003

--2021-10-29 06:54:52--  http://mi.soi.city.ac.uk/datasets/magnatagatune/mp3.zip.001
Resolving mi.soi.city.ac.uk (mi.soi.city.ac.uk)... 138.40.249.126
Connecting to mi.soi.city.ac.uk (mi.soi.city.ac.uk)|138.40.249.126|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://mirg.city.ac.uk/datasets/magnatagatune/mp3.zip.001 [following]
--2021-10-29 06:54:53--  https://mirg.city.ac.uk/datasets/magnatagatune/mp3.zip.001
Resolving mirg.city.ac.uk (mirg.city.ac.uk)... 138.40.249.131
Connecting to mirg.city.ac.uk (mirg.city.ac.uk)|138.40.249.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1100000000 (1.0G) [application/zip]
Saving to: 'mp3.zip.001'

mp3.zip.001         100%[===================>]   1.02G  7.97MB/s    in 2m 35s  

2021-10-29 06:57:30 (6.75 MB/s) - 'mp3.zip.001' saved [1100000000/1100000000]

--2021-10-29 06:57:30--  http://mi.soi.city.ac.uk/datasets/magnatagatune/mp3.zip.002
Resolving mi.soi.city.ac.uk (mi

In [ ]:
!cat mp3.zip.* > mp3_all.zip
!unzip -q mp3_all.zip -d ../mtat_dataset/

In [ ]:
!wget http://mi.soi.city.ac.uk/datasets/magnatagatune/annotations_final.csv -P ../mtat_dataset/
!wget http://mi.soi.city.ac.uk/datasets/magnatagatune/clip_info_final.csv -P ../mtat_dataset/


--2021-10-11 06:32:10--  http://mi.soi.city.ac.uk/datasets/magnatagatune/annotations_final.csv
Resolving mi.soi.city.ac.uk (mi.soi.city.ac.uk)... 138.40.249.126
Connecting to mi.soi.city.ac.uk (mi.soi.city.ac.uk)|138.40.249.126|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://mirg.city.ac.uk/datasets/magnatagatune/annotations_final.csv [following]
--2021-10-11 06:32:11--  https://mirg.city.ac.uk/datasets/magnatagatune/annotations_final.csv
Resolving mirg.city.ac.uk (mirg.city.ac.uk)... 138.40.249.131
Connecting to mirg.city.ac.uk (mirg.city.ac.uk)|138.40.249.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21517373 (21M) [text/csv]
Saving to: ‘mtat_dataset/annotations_final.csv’

annotations_final.c 100%[===================>]  20.52M  6.25MB/s    in 3.3s    

2021-10-11 06:32:16 (6.25 MB/s) - ‘mtat_dataset/annotations_final.csv’ saved [21517373/21517373]

--2021-10-11 06:32:16--  http://mi.soi.city.ac.uk/data

### 0-1 오디오 파일 다운샘플링
- 

## 1. Look at the Dataset

In [6]:
import pandas as pd
from pathlib import Path

data_dir = Path('mtat_dataset/')

In [22]:
meta = pd.read_csv(data_dir / "clip_info_final.csv", delimiter='\t')
labels = pd.read_csv(data_dir / "annotations_final.csv", delimiter='\t')

In [ ]:
meta

,clip_id,track_number,title,artist,album,url,segmentStart,segmentEnd,original_url,mp3_path
0,2,1,BWV54 - I Aria,American Bach Soloists,J.S. Bach Solo Cantatas,http://www.magnatune.com/artists/albums/abs-so...,30,59,http://he3.magnatune.com/all/01--BWV54%20-%20I...,f/american_bach_soloists-j_s__bach_solo_cantat...
1,6,1,BWV54 - I Aria,American Bach Soloists,J.S. Bach Solo Cantatas,http://www.magnatune.com/artists/albums/abs-so...,146,175,http://he3.magnatune.com/all/01--BWV54%20-%20I...,f/american_bach_soloists-j_s__bach_solo_cantat...
2,10,1,BWV54 - I Aria,American Bach Soloists,J.S. Bach Solo Cantatas,http://www.magnatune.com/artists/albums/abs-so...,262,291,http://he3.magnatune.com/all/01--BWV54%20-%20I...,f/american_bach_soloists-j_s__bach_solo_cantat...
3,11,1,BWV54 - I Aria,American Bach Soloists,J.S. Bach Solo Cantatas,http://www.magnatune.com/artists/albums/abs-so...,291,320,http://he3.magnatune.com/all/01--BWV54%20-%20I...,f/american_bach_soloists-j_s__bach_solo_cantat...
4,12,1,BWV54 - I Aria,American Bach Soloists,J.S. Bach Solo Cantatas,http://www.magnatune.com/artists/albums/abs-so...,320,349,http://he3.magnatune.com/all/01--BWV54%20-%20I...,f/american_bach_soloists-j_s__bach_solo_cantat...
...,...,...,...,...,...,...,...,...,...,...
31377,58899,56,La Bressanina,Jacob Heringman,Blame Not My Lute,http://www.magnatune.com/artists/albums/hering...,88,117,http://he3.magnatune.com/all/56-La%20Bressanin...,8/jacob_heringman-blame_not_my_lute-56-la_bres...
31378,58906,57,Lost is my Lyberty,Jacob Heringman,Blame Not My Lute,http://www.magnatune.com/artists/albums/hering...,0,29,http://he3.magnatune.com/all/57-Lost%20is%20my...,8/jacob_heringman-blame_not_my_lute-57-lost_is...
31379,58907,57,Lost is my Lyberty,Jacob Heringman,Blame Not My Lute,http://www.magnatune.com/artists/albums/hering...,30,59,http://he3.magnatune.com/all/57-Lost%20is%20my...,8/jacob_heringman-blame_not_my_lute-57-lost_is...
31380,58908,57,Lost is my Lyberty,Jacob Heringman,Blame Not My Lute,http://www.magnatune.com/artists/albums/hering...,59,88,http://he3.magnatune.com/all/57-Lost%20is%20my...,8/jacob_heringman-blame_not_my_lute-57-lost_is...


In [ ]:
tag = [key for key in labels.keys() if not key in ('clip_id', 'mp3_path')]
len(tag), tag[:10]

(188,
 ['no voice',
  'singer',
  'duet',
  'plucking',
  'hard rock',
  'world',
  'bongos',
  'harpsichord',
  'female singing',
  'clasical'])

In [8]:
# From https://github.com/keunwoochoi/magnatagatune-list
synonyms = [['beat', 'beats'],
			['chant', 'chanting'],
			['choir', 'choral'],
			['classical', 'clasical', 'classic'],
			['drum', 'drums'],
			['electro', 'electronic', 'electronica', 'electric'],
			['fast', 'fast beat', 'quick'],
			['female', 'female singer', 'female singing', 'female vocals', 'female voice', 'woman', 'woman singing', 'women'],
			['flute', 'flutes'],
			['guitar', 'guitars'],
			['hard', 'hard rock'],
			['harpsichord', 'harpsicord'],
			['heavy', 'heavy metal', 'metal'],
			['horn', 'horns'],
			['india', 'indian'],
			['jazz', 'jazzy'],
			['male', 'male singer', 'male vocal', 'male vocals', 'male voice', 'man', 'man singing', 'men'],
			['no beat', 'no drums'],
			['no singer', 'no singing', 'no vocal','no vocals', 'no voice', 'no voices', 'instrumental'],
			['opera', 'operatic'],
			['orchestra', 'orchestral'],
			['quiet', 'silence'],
			['singer', 'singing'],
			['space', 'spacey'],
			['string', 'strings'],
			['synth', 'synthesizer'],
			['violin', 'violins'],
			['vocal', 'vocals', 'voice', 'voices'],
			['strange', 'weird']]

10

In [32]:
np.minimum(labels['beat'].values, 1)

array([0, 0, 0, ..., 0, 0, 0])

In [33]:
labels = pd.read_csv(data_dir / "annotations_final.csv", delimiter='\t')
for syn in synonyms:
  for i in range(1, len(syn)):
    labels[syn[0]] += labels[syn[1]]
  labels = labels.drop(columns=syn[1:])
  labels[syn[0]] = np.minimum(labels['beat'].values, 1)
tag = [key for key in labels.keys() if not key in ('clip_id', 'mp3_path')]
len(tag), tag[:10]
labels

,clip_id,singer,duet,plucking,world,bongos,harpsichord,sitar,chorus,female opera,clarinet,heavy,woodwind,funky,no strings,chimes,foreign,no piano,classical,female,soft rock,eerie,jazz,guitar,quiet,no beat,banjo,solo,folk,wind,happy,ambient,new age,synth,funk,middle eastern,trumpet,percussion,drum,airy,...,orchestra,eastern,old,punk,spanish,sad,sax,slow,male,blues,vocal,no singer,scary,india,rock,dance,piano solo,cello,calm,different,techno,clapping,house,monks,flute,not opera,not english,oriental,beat,upbeat,soft,noise,choir,rap,hip hop,water,baroque,fiddle,english,mp3_path
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
2,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
3,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
4,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25858,58899,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8/jacob_heringman-blame_not_my_lute-56-la_bres...
25859,58906,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,8/jacob_heringman-blame_not_my_lute-57-lost_is...
25860,58907,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8/jacob_heringman-blame_not_my_lute-57-lost_is...
25861,58908,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,8/jacob_heringman-blame_not_my_lute-57-lost_is...


In [1]:
class MTATDataset:
  def __init__(self, dir, top_k=50, min_tag_occur=1, synonyms=(), max_num_data=None, sr=16000, split='train'):
    self.dir = dir
    self.meta = pd.read_csv(data_dir / "clip_info_final.csv", delimiter='\t')
    self.labels = pd.read_csv(data_dir / "annotations_final.csv", delimiter='\t')
    self.sr = sr

    self.get_tag_names()
    self.combine_tags(synonyms)
    self.filter_top_k_tags(top_k)
    self.filter_minimum_tag(min_tag_occur)

    if split=="train":
      sub_dir_ids = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c']
    elif split=='valid':
      sub_dir_ids = ['d']
    else: #test
      sub_dir_ids = ['e', 'f', 'g']

    is_in_set = [True if x[0] in sub_dir_ids else False for x in self.labels['mp3_path'].values.astype('str')]
    self.labels = self.labels.iloc[is_in_set]
    if max_num_data:
      self.labels = self.labels.iloc[:max_num_data]

    self.audios = self.load_audio()
    self.label_tensor = self.convert_label_to_tensor()

  def get_tag_names(self):
    self.tag_names = [key for key in self.labels.keys() if not key in ('clip_id', 'mp3_path')]

  def combine_tags(self, synonyms):
    for syn in synonyms:
      for i in range(1, len(syn)):
        self.labels[syn[0]] += self.labels[syn[i]]
      self.labels = self.labels.drop(columns=syn[1:])
      self.labels[syn[0]] = np.minimum(self.labels[syn[0]].values, 1)
    self.get_tag_names()

  def clip_id_to_tag(self, clip_id):
    return torch.Tensor(self.labels[self.labels['clip_id']==clip_id].values[0,1:-1].astype(int))

  def filter_top_k_tags(self, k=50):
    tag_count = [sum(self.labels[tag]) for tag in self.tag_names]
    descending_sorted_idx = sorted(range(len(tag_count)), key=lambda k: -tag_count[k])
    filtered_tag_names = np.asarray(self.tag_names)[descending_sorted_idx[:k]].tolist()
    for tag in self.tag_names:
      if tag not in filtered_tag_names:
        self.labels = self.labels.drop(columns=tag)
    self.tag_names = filtered_tag_names
  
  def filter_minimum_tag(self, min_tag=1):
    tag_occurs_by_song = np.sum(self.labels[self.tag_names].values, axis=1)
    self.labels = self.labels.drop(index=np.where(np.sum(self.labels[self.tag_names].values, axis=1) < min_tag)[0])

  def load_audio(self):
    audios = []
    fail_idx = []
    for idx in tqdm(range(len(self.labels))):
      try:
        audio, sr = torchaudio.load(self.dir / self.labels.iloc[idx]['mp3_path'])
        if sr != self.sr:
          audio = torchaudio.transforms.Resample(orig_freq=sr, new_freq=self.sr)(audio)
        audios.append(audio)
      except:
        fail_idx.append(idx)
    self.labels = self.labels.drop(index=fail_idx)
    return audios

  def convert_label_to_tensor(self):
    return torch.LongTensor(self.labels.values[1:-1].astype('bool'))

  def __getitem__(self, idx):
    audio = self.audios[idx]
    tag = self.label_tensor[idx]
    return audio, tag

  def __len__(self):
    return len(self.labels)


dataset = MTATDataset(data_dir, synonyms=synonyms, max_num_data=5000)

NameError: ignored

In [72]:
dataset.labels.iloc[0]['mp3_path']

'c/lvx_nova-lvx_nova-01-contimune-30-59.mp3'

In [69]:
dataset.labels.iloc[0]

clip_id                                                 14
singer                                                   0
harpsichord                                              0
sitar                                                    0
heavy                                                    0
foreign                                                  0
no piano                                                 0
classical                                                0
female                                                   0
jazz                                                     0
guitar                                                   0
quiet                                                    0
solo                                                     0
folk                                                     0
ambient                                                  0
new age                                                  0
synth                                                   

In [68]:
labels.iloc[:100]

,clip_id,singer,duet,plucking,world,bongos,harpsichord,sitar,chorus,female opera,clarinet,heavy,woodwind,funky,no strings,chimes,foreign,no piano,classical,female,soft rock,eerie,jazz,guitar,quiet,no beat,banjo,solo,folk,wind,happy,ambient,new age,synth,funk,middle eastern,trumpet,percussion,drum,airy,...,orchestra,eastern,old,punk,spanish,sad,sax,slow,male,blues,vocal,no singer,scary,india,rock,dance,piano solo,cello,calm,different,techno,clapping,house,monks,flute,not opera,not english,oriental,beat,upbeat,soft,noise,choir,rap,hip hop,water,baroque,fiddle,english,mp3_path
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
2,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
3,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
4,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,181,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,e/burning_babylon-stereo_mash_up-01-7_nine_ska...
96,184,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/pain_factor-8_seconds-01-8_seconds-59-88.mp3
97,185,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/pain_factor-8_seconds-01-8_seconds-88-117.mp3
98,186,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/pain_factor-8_seconds-01-8_seconds-117-146.mp3


In [ ]:
from collections import Counter
clip_counter = Counter(labels['clip_id'])

In [ ]:
labels.drop(index=np.where(np.sum(labels[tag].values, axis=1) < 1)[0])

,clip_id,singer,duet,plucking,world,bongos,harpsichord,sitar,chorus,female opera,clarinet,heavy,woodwind,funky,no strings,chimes,foreign,no piano,classical,female,soft rock,eerie,jazz,guitar,quiet,no beat,banjo,solo,folk,wind,happy,ambient,new age,synth,funk,middle eastern,trumpet,percussion,drum,airy,...,orchestra,eastern,old,punk,spanish,sad,sax,slow,male,blues,vocal,no singer,scary,india,rock,dance,piano solo,cello,calm,different,techno,clapping,house,monks,flute,not opera,not english,oriental,beat,upbeat,soft,noise,choir,rap,hip hop,water,baroque,fiddle,english,mp3_path
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
2,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
3,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
4,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25858,58899,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8/jacob_heringman-blame_not_my_lute-56-la_bres...
25859,58906,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,8/jacob_heringman-blame_not_my_lute-57-lost_is...
25860,58907,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8/jacob_heringman-blame_not_my_lute-57-lost_is...
25861,58908,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,8/jacob_heringman-blame_not_my_lute-57-lost_is...


In [ ]:
id_n_path = meta[['clip_id', 'mp3_path']].to_dict('record')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [ ]:
id_n_path[0]['mp3_path']

'f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-30-59.mp3'

In [ ]:
torchaudio.load(id_n_path[0]['mp3_path'])

(tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 5.9605e-08, 5.9605e-08,
          5.9605e-08]]), 16000)

In [ ]:
mp3_path.exists()

True

In [ ]:
SR = 16000
for data_info in id_n_path:
  if not isinstance(data_info['mp3_path'], str):
    continue
  mp3_path = data_dir / data_info['mp3_path']
  if mp3_path.exists():
    try:
      y, sr = torchaudio.load(mp3_path)
    except:
      print(f'Failed to load {str(mp3_path)}')
      continue
    if sr != SR:
      resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=SR)
      y = resampler(y)
    label = labels[labels['clip_id']==data_info['clip_id']].values[0,1:-1].astype(int)
    torch.save({'audio':y, 'tag':torch.LongTensor(label)}, mp3_path.with_suffix('.pt'))


Failed to load mtat_dataset/6/norine_braun-now_and_zen-08-gently-117-146.mp3
Failed to load mtat_dataset/8/jacob_heringman-josquin_des_prez_lute_settings-19-gintzler__pater_noster-204-233.mp3
Failed to load mtat_dataset/9/american_baroque-dances_and_suites_of_rameau_and_couperin-25-le_petit_rien_xiveme_ordre_couperin-88-117.mp3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.mkdir('mtat_dataset')

In [ ]:
for path in Path('.').glob('*'):
  if path in ['mtat_dataset', 'drive']:
    continue
  if path.is_dir() and len(str(path))==1:
    os.rename(path, 'mtat_dataset/'+str(path))

In [ ]:
!zip -r mtat.zip mtat_dataset

In [ ]:
import os
for pt_file in Path('mtat_dataset').rglob('*.pt'):
  os.rename(pt_file, 'mtat_pt/'+pt_file.name)

In [ ]:
test = next(Path('mtat_pt').rglob('*.pt'))

In [ ]:
test.name

'lvx_nova-lvx_nova-05-my_time-552-581.pt'

In [ ]:
!rm -rf mtat_pt/